# 分组级运算和转换

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame, Series

In [2]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.772639,-3.352329
1,a,two,1.533222,1.422569
2,b,one,0.159989,-0.491793
3,b,two,-2.341067,-0.098958
4,a,one,-0.073373,1.279004


In [3]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,1.077496,-0.216919
b,-1.090539,-0.295376


In [4]:
pd.merge(df, k1_means, left_on='key1', right_index=True)


,key1,key2,data1,data2,mean_data1,mean_data2
0,a,one,1.772639,-3.352329,1.077496,-0.216919
1,a,two,1.533222,1.422569,1.077496,-0.216919
4,a,one,-0.073373,1.279004,1.077496,-0.216919
2,b,one,0.159989,-0.491793,-1.090539,-0.295376
3,b,two,-2.341067,-0.098958,-1.090539,-0.295376


In [5]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.033100,1.680563,0.779147,0.399730,0.082306
Steve,1.026243,0.129428,0.899122,-0.106352,0.186312
Wes,-1.117596,1.625927,-0.783645,-0.272723,-0.107360
Jim,-0.790478,-1.929630,1.138231,-0.925137,-2.030326
Travis,-1.843511,-0.765685,-1.317907,-0.415259,0.193282


In [6]:
key = ['one', 'two', 'one', 'two', 'one'] # 每一行的名称
people.groupby(key).mean()

,a,b,c,d,e
one,-0.976002,0.846935,-0.440802,-0.096084,0.056076
two,0.117882,-0.900101,1.018676,-0.515745,-0.922007


In [7]:
people.groupby(key).transform(np.mean) # transform变回原来形状，但是用前面分组聚合的值填空。


,a,b,c,d,e
Joe,-0.976002,0.846935,-0.440802,-0.096084,0.056076
Steve,0.117882,-0.900101,1.018676,-0.515745,-0.922007
Wes,-0.976002,0.846935,-0.440802,-0.096084,0.056076
Jim,0.117882,-0.900101,1.018676,-0.515745,-0.922007
Travis,-0.976002,0.846935,-0.440802,-0.096084,0.056076


In [8]:
def demean(arr):
    return arr - arr.mean()
demeaned = people.groupby(key).transform(demean) # 
demeaned

,a,b,c,d,e
Joe,1.009103,0.833628,1.219948,0.495814,0.026230
Steve,0.908361,1.029529,-0.119555,0.409392,1.108319
Wes,-0.141594,0.778992,-0.342843,-0.176639,-0.163436
Jim,-0.908361,-1.029529,0.119555,-0.409392,-1.108319
Travis,-0.867509,-1.612620,-0.877105,-0.319175,0.137206


In [9]:
demeaned.groupby(key).mean() # 因为前面每个值都减去了平均值，所以应该是0。显示不为0是因为浮点数计算误差。


,a,b,c,d,e
one,-7.401487e-17,1.480297e-16,0.000000e+00,1.850372e-17,0.0
two,0.000000e+00,0.000000e+00,-5.551115e-17,0.000000e+00,0.0


## apply：一般性的“拆分－应用－合并”

In [56]:
tips = pd.read_csv('data/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] # 新加一列，小费与账单金额的比例。
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [11]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:] # 获取小费比例最高的n条数据
top(tips, n=3)

,total_bill,tip,smoker,day,time,size,tip_pct
4,24.59,3.61,No,Sun,Dinner,4,0.146808
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587


In [12]:
tips.groupby('smoker').apply(top, n=2, column='tip_pct') # 先按是否吸烟分组，再分别查看小费比例。

# cc：top这个function的参数就直接放到了apply中

total_bill   tip smoker  day    time  size   tip_pct
smoker                                                        
No     1       10.34  1.66     No  Sun  Dinner     3  0.160542
       2       21.01  3.50     No  Sun  Dinner     3  0.166587

In [13]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')


,,,total_bill,tip,smoker,day,time,size,tip_pct
smoker,day,,,,,,,,
No,Sun,4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [14]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,5.0,0.134633,0.043359,0.059447,0.13978,0.146808,0.160542,0.166587


In [15]:
result.unstack('smoker') # 把列添加到索引，作用在smoker外。


       smoker
count  No        5.000000
mean   No        0.134633
std    No        0.043359
min    No        0.059447
25%    No        0.139780
50%    No        0.146808
75%    No        0.160542
max    No        0.166587
dtype: float64

In [16]:
result.apply(lambda x: x.describe()) # result.describe()


,count,mean,std,min,25%,50%,75%,max
count,1.0,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
mean,5.0,0.134633,0.043359,0.059447,0.13978,0.146808,0.160542,0.166587
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,5.0,0.134633,0.043359,0.059447,0.13978,0.146808,0.160542,0.166587
25%,5.0,0.134633,0.043359,0.059447,0.13978,0.146808,0.160542,0.166587
50%,5.0,0.134633,0.043359,0.059447,0.13978,0.146808,0.160542,0.166587
75%,5.0,0.134633,0.043359,0.059447,0.13978,0.146808,0.160542,0.166587
max,5.0,0.134633,0.043359,0.059447,0.13978,0.146808,0.160542,0.166587


## 禁止分组键

In [17]:
tips.groupby('smoker', group_keys=False).apply(top) # 禁止构成多重索引

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587


In [18]:
# cc：如果不禁止，就构成了多重索引
tips.groupby('smoker').apply(top) # 构成多重索引

total_bill   tip smoker  day    time  size   tip_pct
smoker                                                        
No     0       16.99  1.01     No  Sun  Dinner     2  0.059447
       3       23.68  3.31     No  Sun  Dinner     2  0.139780
       4       24.59  3.61     No  Sun  Dinner     4  0.146808
       1       10.34  1.66     No  Sun  Dinner     3  0.160542
       2       21.01  3.50     No  Sun  Dinner     3  0.166587

## 分位数和桶分析

In [19]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})
factor = pd.cut(frame.data1, 4) # 切4份
factor[:5] # 前5个元素所在区间

0    (-1.435, 0.00305]
1    (-1.435, 0.00305]
2     (0.00305, 1.442]
3    (-1.435, 0.00305]
4     (0.00305, 1.442]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.88, -1.435] < (-1.435, 0.00305] < (0.00305, 1.442] < (1.442, 2.88]]

In [20]:
def get_stats(group):
    return {'min': group.min(),
             'max': group.max(),
             'count': group.count(),
             'mean': group.mean()}
grouped = frame.data2.groupby(factor) # 根据data1的分段，对data2进行分组。
# cc：首先，到这一步的用法就很有用了！
# factor 是创建好的区间，虽然没有在frame中，但是可以按照factor进行分组；

result = grouped.apply(get_stats)
print(result)
print(result.unstack())

data1                   
(-2.88, -1.435]    min       -2.351596
                   max        2.521797
                   count     72.000000
                   mean       0.163807
(-1.435, 0.00305]  min       -2.674210
                   max        2.556429
                   count    427.000000
                   mean      -0.030455
(0.00305, 1.442]   min       -2.975735
                   max        2.424534
                   count    436.000000
                   mean       0.005998
(1.442, 2.88]      min       -2.574812
                   max        2.691533
                   count     65.000000
                   mean       0.103121
Name: data2, dtype: float64
                        min       max  count      mean
data1                                                 
(-2.88, -1.435]   -2.351596  2.521797   72.0  0.163807
(-1.435, 0.00305] -2.674210  2.556429  427.0 -0.030455
(0.00305, 1.442]  -2.975735  2.424534  436.0  0.005998
(1.442, 2.88]     -2.574812  2.691533   65.0  0.

In [21]:
grouping = pd.qcut(frame.data1, 10, labels=list('ABCDEFGHIJ')) # False的话就是默认数字编号
grouping.head() # 返回每个元素的区间编号

0    E
1    E
2    H
3    A
4    G
Name: data1, dtype: category
Categories (10, object): ['A' < 'B' < 'C' < 'D' ... 'G' < 'H' < 'I' < 'J']

In [22]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
A,-2.351596,2.521797,100.0,0.018743
B,-2.637367,2.556429,100.0,0.045279
C,-2.674210,2.040353,100.0,0.126277
D,-2.497992,1.999256,100.0,-0.206787
E,-2.398990,2.518462,100.0,0.005545
F,-2.180804,2.424534,100.0,0.085148
G,-2.207227,2.367427,100.0,0.063048
H,-2.975735,2.044182,100.0,-0.141416
I,-2.054868,2.339424,100.0,0.081223


## 用特定于分组的值填充缺失值

In [23]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s.fillna(s.mean()) # 用平均数填充缺失值

0    0.691767
1    0.685475
2    0.691767
3    0.666042
4    0.691767
5    0.723786
dtype: float64

In [24]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4 # 前4个东部州，后4个西部州。
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.057784
New York      0.975776
Vermont            NaN
Florida       1.087341
Oregon       -0.609988
Nevada             NaN
California   -0.259613
Idaho              NaN
dtype: float64

In [25]:
data.groupby(group_key).mean() # 非NA求平均值


East    0.706967
West   -0.434800
dtype: float64

In [26]:
fill_mean = lambda g: g.fillna(g.mean()) # cc：fill_mean 为一个function
data.groupby(group_key).apply(fill_mean) # 分组之后用每组的平均值填充

Ohio          0.057784
New York      0.975776
Vermont       0.706967
Florida       1.087341
Oregon       -0.609988
Nevada       -0.434800
California   -0.259613
Idaho        -0.434800
dtype: float64

In [28]:
fill_values = {'East': 0.5, 'West': -1} # 指定填充值
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.057784
New York      0.975776
Vermont       0.500000
Florida       1.087341
Oregon       -0.609988
Nevada       -1.000000
California   -0.259613
Idaho        -1.000000
dtype: float64

## 随机采样和排列

In [30]:
# 红桃（Hearts）、黑桃（Spades）、梅花（Clubs）、方片（Diamonds）
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4 # Python3下range是生成器，必须用list显示展开。
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = Series(card_val, index=cards)
deck.head() # 牌面数组

AH    1
2H    2
3H    3
4H    4
5H    5
dtype: int64

In [33]:
def draw(deck, n=5): # 随机抽n张
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

# cc：又遇到这个写法了：np.random.permutation(len(deck))，作用是：随机排列一个序列，返回一个排列的序列

6H     6
5S     5
4S     4
JD    10
KD    10
dtype: int64

In [34]:
# 每种花色中随机抽取两张牌
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2) # 默认根据索引排序，索引的最后一个字符是花色。

# cc：这也太。。。厉害了！？？？？

C  8C      8
   7C      7
D  2D      2
   5D      5
H  JH     10
   9H      9
S  JS     10
   10S    10
dtype: int64

In [38]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2) # 效果一样，但是不用多重索引。


6C      6
QC     10
6D      6
KD     10
6H      6
10H    10
8S      8
3S      3
dtype: int64

## 分组加权平均数和相关系数

In [39]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-2.426934,0.090998
1,a,0.654896,0.808360
2,a,0.407956,0.888513
3,a,0.888946,0.036545
4,b,-0.828638,0.376539
5,b,1.090013,0.612026
6,b,0.581775,0.769727
7,b,-1.070719,0.051034


In [40]:
grouped = df.groupby('category')


In [41]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights']) # 求加权平均，weights自动归一化处理。
grouped.apply(get_wavg) # 分组计算

category
a    0.385607
b    0.413561
dtype: float64

In [50]:
close_px = pd.read_csv('data/stock_px.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5472 entries, 1990-02-01 to 2011-10-14
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AA      5472 non-null   float64
 1   AAPL    5472 non-null   float64
 2   GE      5472 non-null   float64
 3   IBM     5472 non-null   float64
 4   JNJ     5472 non-null   float64
 5   MSFT    5472 non-null   float64
 6   PEP     5471 non-null   float64
 7   SPX     5472 non-null   float64
 8   XOM     5472 non-null   float64
dtypes: float64(9)
memory usage: 427.5 KB


In [51]:
close_px.head()


,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-01,4.98,7.86,2.87,16.79,4.27,0.51,6.04,328.79,6.12
1990-02-02,5.04,8.00,2.87,16.89,4.37,0.51,6.09,330.92,6.24
1990-02-05,5.07,8.18,2.87,17.32,4.34,0.51,6.05,331.85,6.25
1990-02-06,5.01,8.12,2.88,17.56,4.32,0.51,6.15,329.66,6.23
1990-02-07,5.04,7.77,2.91,17.93,4.38,0.51,6.17,333.75,6.33


In [52]:
close_px.pct_change()
# cc：
# 表示当前元素与先前元素的相差百分比，当然指定periods=n,表示当前元素与先前n 个元素的相差百分比。

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-02-02,0.012048,0.017812,0.000000,0.005956,0.023419,0.000000,0.008278,0.006478,0.019608
1990-02-05,0.005952,0.022500,0.000000,0.025459,-0.006865,0.000000,-0.006568,0.002810,0.001603
1990-02-06,-0.011834,-0.007335,0.003484,0.013857,-0.004608,0.000000,0.016529,-0.006599,-0.003200
1990-02-07,0.005988,-0.043103,0.010417,0.021071,0.013889,0.000000,0.003252,0.012407,0.016051
...,...,...,...,...,...,...,...,...,...
2011-10-10,0.039135,0.051406,0.041290,0.023192,0.020592,0.026286,0.013930,0.034125,0.036977
2011-10-11,0.020813,0.029526,0.000000,-0.008681,-0.007295,0.002227,-0.014870,0.000544,-0.000131
2011-10-12,-0.024272,0.004747,0.016109,0.006054,0.005785,-0.001481,0.028712,0.009795,0.011669
2011-10-13,0.004975,0.015515,-0.010976,0.003761,-0.001554,0.008160,-0.005423,-0.002974,-0.010238


In [53]:
rets = close_px.pct_change().dropna() # 扔掉有空数据的行
spx_corr = lambda x: x.corrwith(x['SPX']) # 与SPX的相关系数
by_year = rets.groupby(lambda x: x.year) # 指定用那个函数去做group
by_year.apply(spx_corr) # 按年分组并计算与SPX的相关系数

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990,0.595024,0.545067,0.752187,0.738361,0.801145,0.586691,0.783168,1.0,0.517586
1991,0.453574,0.365315,0.759607,0.557046,0.646401,0.524225,0.641775,1.0,0.569335
1992,0.398180,0.498732,0.632685,0.262232,0.515740,0.492345,0.473871,1.0,0.318408
1993,0.259069,0.238578,0.447257,0.211269,0.451503,0.425377,0.385089,1.0,0.318952
1994,0.428549,0.268420,0.572996,0.385162,0.372962,0.436585,0.450516,1.0,0.395078
1995,0.291532,0.161829,0.519126,0.416390,0.315733,0.453660,0.413144,1.0,0.368752
1996,0.292344,0.191482,0.750724,0.388497,0.569232,0.564015,0.421477,1.0,0.538736
1997,0.564427,0.211435,0.827512,0.646823,0.703538,0.606171,0.509344,1.0,0.695653
1998,0.533802,0.379883,0.815243,0.623982,0.591988,0.698773,0.494213,1.0,0.369264
1999,0.099033,0.425584,0.710928,0.486167,0.517061,0.631315,0.336593,1.0,0.315383


In [54]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT'])) # 计算两个股票之间的相关系数


1990    0.408271
1991    0.266807
1992    0.450592
1993    0.236917
1994    0.361638
1995    0.258642
1996    0.147539
1997    0.196144
1998    0.364106
1999    0.329484
2000    0.275298
2001    0.563156
2002    0.571435
2003    0.486262
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 面向分组的线性回归

In [55]:
def regress(data, yvar, xvars):
    y= data[yvar]
    x = data[xvars]
    x['intercept'] = 1.
    result = sm.OLS(y, x).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
1990,1.512772,0.001395
1991,1.187351,0.000396
1992,1.832427,0.000164
1993,1.390470,-0.002657
1994,1.190277,0.001617
1995,0.858818,-0.001423
1996,0.829389,-0.001791
1997,0.749928,-0.001901
1998,1.164582,0.004075
1999,1.384989,0.003273
